In [1]:
import pandas as pd

## Assemble PacBio reads with Canu:

```canu -p ecoli -d ecoli-pacbio4 stopOnLowCoverage=6 genomeSize=4.8m useGrid=false -pacbio-raw ../../NGS/6/pacbio_10x.fq.gz ```

## Assemble Illumina reads with SPAdes:

```spades.py -k 21,33,55,77 --careful --pe1-1 ../../NGS/6/illumina.100x.1.fq.gz --pe1-2 ../../NGS/6/illumina.100x.2.fq.gz -o illumina_assembly```

## Hybrid assembly with SPAdes:

```spades.py -k 77 --trusted-contigs ecoli-pacbio2/ecoli.contigs.fasta --pe1-1 ../../NGS/6/illumina.100x.1.fq.gz --pe1-2 ../../NGS/6/illumina.100x.2.fq.gz -o hybrid_20```

# QUAST assembly evaluation

In [2]:
quast_report = pd.read_csv('quast_results/latest/report.tsv', delimiter='\t')

## Basic statistics

### Contig sizes

In [3]:
quast_report.where(quast_report.Assembly.isin(['# contigs',
                                               '# contigs (>= 1000 bp)',
                                               'Largest contig',
                                               'Total length'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
1,# contigs (>= 1000 bp),132,24,3,1,73,6,3,2,2
12,# contigs,132,24,3,1,81,8,4,3,3
13,Largest contig,180610,749388,4640801,4652986,315011,1999155,4589664,4589665,4589665
14,Total length,3817958,4606872,4656609,4652986,4561340,4638678,4639164,4641289,4641289


![gc1](pics/cumulative_plot.png)

### Nx and Lx

In [4]:
quast_report.where(quast_report.Assembly.isin(['N50',
                                               'N75',
                                               'L50',
                                               'L75'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
18,N50,47884,273158,4640801,4652986,133059,1034546,4589664,4589665,4589665
20,N75,26677,199297,4640801,4652986,66190,552505,4589664,4589665,4589665
22,L50,27,5,1,1,12,2,1,1,1
24,L75,52,10,1,1,24,3,1,1,1


![gc1](pics/Nx_plot.png)

### GC content and Ns

In [5]:
quast_report.where(quast_report.Assembly.isin(['GC (%)',
                                               '# N\'s per 100 kbp'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
16,GC (%),50.50,50.76,50.76,50.79,50.74,50.78,50.78,50.79,50.79
37,# N's per 100 kbp,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### _По базовым статистикам сборка прочтений pacbio с максимальным покрытием оказалась самой лучшей - потому что они собрались в один контиг._

## Reference-based metrics

### NGx and LGx

In [6]:
quast_report.where(quast_report.Assembly.isin(['NG50',
                                               'NG75',
                                               'LG50',
                                               'LG75'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
19,NG50,42731,273158,4640801,4652986,133059,1034546,4589664,4589665,4589665
21,NG75,12082,199297,4640801,4652986,64443,552505,4589664,4589665,4589665
23,LG50,36,5,1,1,12,2,1,1,1
25,LG75,85,10,1,1,25,3,1,1,1


### Alignment statistics

In [7]:
quast_report.where(quast_report.Assembly.isin(['Genome fraction (%)',
                                               'Total length','Reference length',
                                               'Duplication ratio',
                                               'Largest alignment', '# mismatches per 100 kbp',
                                               'Total aligned length', '# indels per 100 kbp'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
14,Total length,3817958,4606872,4656609,4652986,4561340,4638678,4639164,4641289,4641289
15,Reference length,4639675,4639675,4639675,4639675,4639675,4639675,4639675,4639675,4639675
35,Genome fraction (%),82.135,98.615,99.994,99.998,98.271,99.904,99.916,99.962,99.962
36,Duplication ratio,1.002,1.007,1.004,1.003,1.000,1.001,1.001,1.001,1.001
38,# mismatches per 100 kbp,48.81,6.05,1.23,0.39,1.16,10.20,9.15,9.62,9.66
39,# indels per 100 kbp,677.23,151.42,34.16,11.29,0.37,1.38,0.73,0.88,0.88
40,Largest alignment,149583,514798,3022595,3026092,315011,1034546,3023576,3023577,3023577
41,Total aligned length,3817753,4606854,4653329,4652984,4560081,4637935,4635836,4637961,4637961


### _По статистикам, основанным на референсе, у сборки прочтений illumina оказалось меньше ошибок в выравнивании, а у сборки pacbio больше процент покрытого генома. Их гибридная сборка, соответственно, дает в результате увеличение в проценте покрытого генома и улучшение в качестве. _

### Missassemblies

In [8]:
quast_report.where(quast_report.Assembly.isin(['# misassemblies',
                                               '# misassembled contigs',
                                               'Misassembled contigs length',
                                               '# local misassemblies'])).dropna()

,Assembly,pacbio10x,pacbio20x,pacbio40x,pacbio80x,illuminaK77,hybrid10,hybrid20,hybrid40,hybrid80
26,# misassemblies,11,8,8,8,0,6,6,6,6
27,# misassembled contigs,7,3,1,1,0,2,1,1,1
28,Misassembled contigs length,345889,1187160,4640801,4652986,0,2499203,4589664,4589665,4589665
29,# local misassemblies,1,2,2,2,9,6,5,5,5


### _По misassemblies прочтения illumina оказались лучше. Возможно, в pacbio они появились из-за того, что весь геном собрался в один контиг, и поэтому, в случае повторов или длинных мономеров неправильное разрешение всегда ведет к мисассемблу - а прочтения illumina в таких местах разделяются на разные контиги и выравниваются куда надо_